In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!pip install yfinance --upgrade --no-cache-dir

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=bc7ca1b36705ae9a217d3555c9364b3d175434097680038a82cccd3cec5782b3
  Stored in directory: /tmp/pip-ephem-wheel-cache-gkrpdwoj/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split 
from itertools import permutations, combinations

# 모듈가져오기
import warnings
# 경고 무시 처리
warnings.filterwarnings('ignore')

# 시간처리
from datetime import datetime
from pandas_datareader import data
import yfinance as yf
yf.pdr_override()
from sklearn.metrics import r2_score

In [ ]:
# KOSPI TOP10종목 데이터 불러오기
# /content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/stock_top10.csv
path = '/content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/stock_top10.csv'
stock_top10 = pd.read_csv('/content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/stock_top10.csv', encoding='utf-8')
stock_top10

,Date,Adj Close,name,code,dollar,Dubai,Brent,Oman,WTI
0,2020-02-14,28500.0,일진다이아,81000,1181.5,55.23,57.32,55.73,52.05
1,2020-02-18,28300.0,일진다이아,81000,1182.2,55.11,57.75,55.56,52.05
2,2020-02-19,27950.0,일진다이아,81000,1188.1,56.39,59.12,56.87,53.29
3,2020-02-20,29350.0,일진다이아,81000,1190.8,56.92,59.31,57.52,53.78
4,2020-02-21,28400.0,일진다이아,81000,1197.0,56.41,58.50,57.30,53.38
...,...,...,...,...,...,...,...,...,...
1055,2020-07-16,11200.0,드림텍,192650,1200.7,43.37,43.37,43.63,40.75
1056,2020-07-17,11250.0,드림텍,192650,1204.1,42.86,43.14,43.16,40.59
1057,2020-07-20,11300.0,드림텍,192650,1205.7,42.72,43.28,42.87,40.81
1058,2020-07-21,11950.0,드림텍,192650,1203.7,43.87,44.32,44.02,41.96


In [ ]:
top10 = list()
for i in range(10):
  top10.append(stock_top10[i*106:(1+i)*106])
top10[0]

,Date,Adj Close,name,code,dollar,Dubai,Brent,Oman,WTI
0,2020-02-14,28500.0,일진다이아,81000,1181.5,55.23,57.32,55.73,52.05
1,2020-02-18,28300.0,일진다이아,81000,1182.2,55.11,57.75,55.56,52.05
2,2020-02-19,27950.0,일진다이아,81000,1188.1,56.39,59.12,56.87,53.29
3,2020-02-20,29350.0,일진다이아,81000,1190.8,56.92,59.31,57.52,53.78
4,2020-02-21,28400.0,일진다이아,81000,1197.0,56.41,58.50,57.30,53.38
...,...,...,...,...,...,...,...,...,...
101,2020-07-16,57200.0,일진다이아,81000,1200.7,43.37,43.37,43.63,40.75
102,2020-07-17,56600.0,일진다이아,81000,1204.1,42.86,43.14,43.16,40.59
103,2020-07-20,59000.0,일진다이아,81000,1205.7,42.72,43.28,42.87,40.81
104,2020-07-21,59300.0,일진다이아,81000,1203.7,43.87,44.32,44.02,41.96


In [ ]:
for i in range(10):
  print(top10[i].name.unique()[0], top10[i].code.unique()[0])

일진다이아 81000
디피씨 26890
효성중공업 298040
JW중외제약우 1065
코오롱머티리얼 144620
현대건설우 725
유니온머티리얼 47400
두산2우B 157
모나미 5360
드림텍 192650


In [ ]:
start_date = '2010-02-14'#'2010-01-01'
end_date = '2020-02-13'


code = ['081000.ks', '026890.ks', '298040.ks', '001065.ks', '144620.ks', '000725.ks', '047400.ks', '000157.ks', '005360.ks', '192650.ks']
tmp = list()
for i in range(len(top10)):
  TRADE_CODE = code[i]
  tmp1 = data.get_data_yahoo( TRADE_CODE, start_date, end_date)
  if len(tmp1) > 2300:
    tmp.append(tmp1)
    print(i)
  

# 일진다이아 081000
# 디피씨 026890
# JW중외제약우 001065
# 현대건설우 000725
# 유니온머티리얼 047400
# 두산2우B 000157
# 모나미 005360

[*********************100%***********************]  1 of 1 completed
0
[*********************100%***********************]  1 of 1 completed
1
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
3
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
5
[*********************100%***********************]  1 of 1 completed
6
[*********************100%***********************]  1 of 1 completed
7
[*********************100%***********************]  1 of 1 completed
8
[*********************100%***********************]  1 of 1 completed


In [ ]:
top10_10years = list()
for i in range(len(tmp)):
  top10_10years.append(tmp[i][['Close']])

KeyError: ignored

In [ ]:
for i in range(len(top10_10years)):
  print(top10_10years[i].shape)

In [ ]:
# 0번째 : 02-16 ~ 02-13 
# 1번째 : 02-16 ~ 02-13
# 2번째 : 02-16 ~ 02-13
# 3번째 : 02-17 ~ 02-13
# 4번째 : 02-16 ~ 02-13
# 5번째 : 02-16 ~ 02-13
# 6번째 : 02-16 ~ 02-13
top10_10years[5]

,Close
Date,
2010-02-16,39810.199219
2010-02-17,40909.898438
2010-02-18,42779.398438
2010-02-19,40854.898438
2010-02-22,41789.699219
...,...
2020-02-07,58000.000000
2020-02-10,57200.000000
2020-02-11,57500.000000


In [ ]:
len(top10_10years)

7

In [ ]:
for i in range(7):
  tmp3 = pd.Series (top10_10years[i].index)
  date_str = tmp3.apply( lambda x:str(x)[:10] )
  top10_10years[i]['Date'] = date_str.values
  top10_10years[i].set_index(['Date'], inplace=True)

top10_10years[0].index

Index(['2010-02-16', '2010-02-17', '2010-02-18', '2010-02-19', '2010-02-22',
       '2010-02-23', '2010-02-24', '2010-02-25', '2010-02-26', '2010-03-02',
       ...
       '2020-01-31', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06',
       '2020-02-07', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13'],
      dtype='object', name='Date', length=2451)

In [ ]:
# 날짜를 리스트로 교집합 구하기 
date = top10_10years[0].index.unique() & top10_10years[1].index.unique()& top10_10years[2].index.unique()& top10_10years[3].index.unique()& top10_10years[4].index.unique()& top10_10years[5].index.unique()& top10_10years[6].index.unique()
date.shape

(2350,)

In [ ]:
top10 = list()
for i in range(len(top10_10years)):
  top10.append(top10_10years[i].loc[date, :])

In [ ]:
for i in range(len(top10)):
  print(top10[i].shape)

(2350, 1)
(2350, 1)
(2350, 1)
(2350, 1)
(2350, 1)
(2350, 1)
(2350, 1)


In [ ]:
tmp = pd.Series (top10[0].index)
date_str = tmp.apply( lambda x:str(x)[:10] )
for i in range(len(top10)):
  top10[i]['Date'] = date_str.values
  top10[i].set_index(['Date'], inplace=True)

In [ ]:
top10[0].index.unique()

Index(['2010-02-17', '2010-02-18', '2010-02-19', '2010-02-22', '2010-02-23',
       '2010-02-24', '2010-02-25', '2010-02-26', '2010-03-02', '2010-03-03',
       ...
       '2020-01-31', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06',
       '2020-02-07', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13'],
      dtype='object', name='Date', length=2350)

In [ ]:
###############################################
# 환율데이터 불러오기 및 전처리
# /content/drive/My Drive/데이터분석 프로젝트/data/krw_usd.xlsx
path = '/content/drive/My Drive/데이터분석 프로젝트/data/krw_usd.xlsx'
tmp_dollar = pd.read_excel(path, encoding='utf-8')
tmp_dollar

,ÀÏÀÚ,¼Û±Ý,Unnamed: 2,ÇöÂû,Unnamed: 4,¸Å¸Å±âÁØÀ²,±âÁØÈ¯À²,´ë¹ÌÈ¯»êÀ²
0,NaN,º¸³»½Ç¶§,¹ÞÀ¸½Ç¶§,»ç½Ç¶§,ÆÄ½Ç¶§,(ÀüÀÏ´ëºñ),NaN,NaN
1,2020.02.13,1192.9,1170.1,1202.17,1160.83,"1,181.50»ó½Â2.00",1179.8,1.0
2,2020.02.12,1190.9,1168.1,1200.14,1158.86,"1,179.50ÇÏ¶ô-4.50",1184.1,1.0
3,2020.02.11,1195.4,1172.6,1204.72,1163.28,"1,184.00ÇÏ¶ô-4.00",1189.8,1.0
4,2020.02.10,1199.5,1176.5,1208.79,1167.21,"1,188.00ÇÏ¶ô-3.00",1187.2,1.0
...,...,...,...,...,...,...,...,...
2471,2010.02.22,1159.6,1137.4,1168.59,1128.41,"1,148.50ÇÏ¶ô-8.50",1157.8,1.0
2472,2010.02.19,1168.2,1145.8,1177.24,1136.76,"1,157.00»ó½Â6.50",1147.5,1.0
2473,2010.02.18,1161.6,1139.4,1170.63,1130.37,"1,150.50»ó½Â7.50",1142.7,1.0
2474,2010.02.17,1154,1132,1163,1123,"1,143.00ÇÏ¶ô-7.00",1152.9,1.0


In [ ]:
tmp_dollar.columns

Index(['ÀÏÀÚ', '¼Û±Ý', 'Unnamed: 2', 'ÇöÂû', 'Unnamed: 4', '¸Å¸Å±âÁØÀ²',
       '±âÁØÈ¯À²', '´ë¹ÌÈ¯»êÀ²'],
      dtype='object')

In [ ]:
dollar = tmp_dollar[['ÀÏÀÚ', '±âÁØÈ¯À²']]

In [ ]:
dollar.columns = ['Date', 'dollar']

In [ ]:
dollar.sort_values('Date', ascending=True, inplace=True)

In [ ]:
dollar.set_index(['Date'], inplace=True)

In [ ]:
dollar.dropna(axis=0, inplace=True)
dollar

,dollar
Date,
2010.02.16,1154.9
2010.02.17,1152.9
2010.02.18,1142.7
2010.02.19,1147.5
2010.02.22,1157.8
...,...
2020.02.07,1181.4
2020.02.10,1187.2
2020.02.11,1189.8


In [ ]:
dollar

,dollar
Date,
2010.02.16,1154.9
2010.02.17,1152.9
2010.02.18,1142.7
2010.02.19,1147.5
2010.02.22,1157.8
...,...
2020.02.07,1181.4
2020.02.10,1187.2
2020.02.11,1189.8


In [ ]:
# 종목의 날짜와 달러데이터프레임의 날짜 수를 맞춰준다.
top10[0].index.unique() & dollar.index.unique()

Index([], dtype='object', name='Date')

In [ ]:
dollar.index.unique()

Index(['2010.02.16', '2010.02.17', '2010.02.18', '2010.02.19', '2010.02.22',
       '2010.02.23', '2010.02.24', '2010.02.25', '2010.02.26', '2010.03.02',
       ...
       '2020.01.31', '2020.02.03', '2020.02.04', '2020.02.05', '2020.02.06',
       '2020.02.07', '2020.02.10', '2020.02.11', '2020.02.12', '2020.02.13'],
      dtype='object', name='Date', length=2475)

In [ ]:
dollar.index

Index(['2010.02.16', '2010.02.17', '2010.02.18', '2010.02.19', '2010.02.22',
       '2010.02.23', '2010.02.24', '2010.02.25', '2010.02.26', '2010.03.02',
       ...
       '2020.01.31', '2020.02.03', '2020.02.04', '2020.02.05', '2020.02.06',
       '2020.02.07', '2020.02.10', '2020.02.11', '2020.02.12', '2020.02.13'],
      dtype='object', name='Date', length=2475)

In [ ]:
# dollar데이터프레임의 날짜 형식이 다르기 때문에 전처리 해준다
tmp = dollar.index
dollar['Date'] = tmp
dollar['Date']

Date
2010.02.16    2010.02.16
2010.02.17    2010.02.17
2010.02.18    2010.02.18
2010.02.19    2010.02.19
2010.02.22    2010.02.22
                 ...    
2020.02.07    2020.02.07
2020.02.10    2020.02.10
2020.02.11    2020.02.11
2020.02.12    2020.02.12
2020.02.13    2020.02.13
Name: Date, Length: 2475, dtype: object

In [ ]:
dollar['Date'] = dollar['Date'].apply(lambda x : x.replace('.', '-'))
dollar.set_index(['Date'], inplace=True)
dollar

,dollar
Date,
2010-02-16,1154.9
2010-02-17,1152.9
2010-02-18,1142.7
2010-02-19,1147.5
2010-02-22,1157.8
...,...
2020-02-07,1181.4
2020-02-10,1187.2
2020-02-11,1189.8


In [ ]:
####################################################################################################
# 10년 금리 데이터 불러오기
# /content/drive/My Drive/데이터분석 프로젝트/data/interest_rate.csv
path = '/content/drive/My Drive/데이터분석 프로젝트/data/interest_rate_10years.csv'
interest = pd.read_csv(path, encoding='utf-8')
interest

,통계표,4.1.1 시장금리(일별)
0,항목명1,"콜금리(1일, 전체거래)"
1,단위,연%
2,변환,원자료
3,2010/02/14,NaN
4,2010/02/15,NaN
...,...,...
3650,2020/02/09,NaN
3651,2020/02/10,1.24
3652,2020/02/11,1.26
3653,2020/02/12,1.26


In [ ]:
interest.columns=['Date', 'interest_rate']

In [ ]:
interest.dropna(axis=0, inplace=True)

In [ ]:
interest.shape

(2478, 2)

In [ ]:
interest

,Date,interest_rate
0,항목명1,"콜금리(1일, 전체거래)"
1,단위,연%
2,변환,원자료
5,2010/02/16,2.01
6,2010/02/17,2.01
...,...,...
3648,2020/02/07,1.22
3651,2020/02/10,1.24
3652,2020/02/11,1.26
3653,2020/02/12,1.26


In [ ]:
interest.drop([0, 1, 2], inplace=True)
interest

,Date,interest_rate
5,2010/02/16,2.01
6,2010/02/17,2.01
7,2010/02/18,2.01
8,2010/02/19,2.01
11,2010/02/22,1.98
...,...,...
3648,2020/02/07,1.22
3651,2020/02/10,1.24
3652,2020/02/11,1.26
3653,2020/02/12,1.26


In [ ]:
interest["Date"] = interest["Date"].apply(lambda x: x.replace('/', '-'))
interest["Date"]

5       2010-02-16
6       2010-02-17
7       2010-02-18
8       2010-02-19
11      2010-02-22
           ...    
3648    2020-02-07
3651    2020-02-10
3652    2020-02-11
3653    2020-02-12
3654    2020-02-13
Name: Date, Length: 2475, dtype: object

In [ ]:
interest.set_index(['Date'], inplace=True)

In [ ]:
interest

,interest_rate
Date,
2010-02-16,2.01
2010-02-17,2.01
2010-02-18,2.01
2010-02-19,2.01
2010-02-22,1.98
...,...
2020-02-07,1.22
2020-02-10,1.24
2020-02-11,1.26


In [ ]:
#################################################################
# 유가 데이터 불러오기
# 유가데이터는 1년단위로만 데이터를 가져올 수 있어서 10개의csv파일을 합쳐줘야 한다
path = '/content/drive/My Drive/데이터분석 프로젝트/data/유가'
col = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
tmp2 = list()
for i in range(11):
  tmp2.append(pd.read_csv(f'{path}/{col[i]}.csv', encoding='utf-8'))


In [ ]:
tmp2[0][:-5]

,﻿일,유종,WTI
0,02월 16일,NaN,77.01
1,02월 17일,NaN,77.33
2,02월 18일,NaN,79.06
3,02월 19일,NaN,79.81
4,02월 22일,NaN,80.16
...,...,...,...
218,12월 27일,NaN,91.00
219,12월 28일,NaN,91.49
220,12월 29일,NaN,91.12
221,12월 30일,NaN,89.84


In [ ]:
for i in range(11):
  del tmp2[i]['유종']
  tmp2[i].columns = ['Date', 'WTI']

In [ ]:
for i in range(11):
  tmp2[i] = tmp2[i][:-5]

In [ ]:
for i in range(11):
  tmp2[i]['Date'] = tmp2[i]['Date'].apply(lambda x : f'2010-{x}')

In [ ]:
tmp2[0]['Date'].values

array(['2010-02월\xa016일', '2010-02월\xa017일', '2010-02월\xa018일',
       '2010-02월\xa019일', '2010-02월\xa022일', '2010-02월\xa023일',
       '2010-02월\xa024일', '2010-02월\xa025일', '2010-02월\xa026일',
       '2010-03월\xa001일', '2010-03월\xa002일', '2010-03월\xa003일',
       '2010-03월\xa004일', '2010-03월\xa005일', '2010-03월\xa008일',
       '2010-03월\xa009일', '2010-03월\xa010일', '2010-03월\xa011일',
       '2010-03월\xa012일', '2010-03월\xa015일', '2010-03월\xa016일',
       '2010-03월\xa017일', '2010-03월\xa018일', '2010-03월\xa019일',
       '2010-03월\xa022일', '2010-03월\xa023일', '2010-03월\xa024일',
       '2010-03월\xa025일', '2010-03월\xa026일', '2010-03월\xa029일',
       '2010-03월\xa030일', '2010-03월\xa031일', '2010-04월\xa001일',
       '2010-04월\xa005일', '2010-04월\xa006일', '2010-04월\xa007일',
       '2010-04월\xa008일', '2010-04월\xa009일', '2010-04월\xa012일',
       '2010-04월\xa013일', '2010-04월\xa014일', '2010-04월\xa015일',
       '2010-04월\xa016일', '2010-04월\xa019일', '2010-04월\xa020일',
       '2010-04월\xa021일', '2010-04월\xa02

In [ ]:
tmp2[0]['Date'] = tmp2[0]['Date'].apply(lambda x : f'2010-{x}')
tmp2[0]

,Date,WTI
0,2010-2010-02월 16일,77.01
1,2010-2010-02월 17일,77.33
2,2010-2010-02월 18일,79.06
3,2010-2010-02월 19일,79.81
4,2010-2010-02월 22일,80.16
...,...,...
218,2010-2010-12월 27일,91.00
219,2010-2010-12월 28일,91.49
220,2010-2010-12월 29일,91.12
221,2010-2010-12월 30일,89.84


In [ ]:
year = 2011
for i in range(1, 11):
  tmp2[i]['Date'] = tmp2[i]['Date'].apply(lambda x : f'{year}-{x}')
  year= year+1

In [ ]:
year = 2011
for i in range(1, 11):
  tmp2[i]['Date'] = tmp2[i]['Date'].apply(lambda x : x.replace('\xa0', ''))
  year= year+1

In [ ]:
tmp2[9].shape

(252, 2)

In [ ]:
oil_10years = pd.concat([tmp2[0], tmp2[1], tmp2[2], tmp2[3], tmp2[4], tmp2[5], tmp2[6], tmp2[7], tmp2[8], tmp2[9], tmp2[10]])
oil_10years

,Date,WTI
0,2010-2010-02월 16일,77.01
1,2010-2010-02월 17일,77.33
2,2010-2010-02월 18일,79.06
3,2010-2010-02월 19일,79.81
4,2010-2010-02월 22일,80.16
...,...,...
25,2020-2010-02월 07일,50.32
26,2020-2010-02월 10일,49.57
27,2020-2010-02월 11일,49.94
28,2020-2010-02월 12일,51.17


In [ ]:
oil_10years.set_index(['Date'], inplace=True)
oil_10years

,WTI
Date,
2010-2010-02월 16일,77.01
2010-2010-02월 17일,77.33
2010-2010-02월 18일,79.06
2010-2010-02월 19일,79.81
2010-2010-02월 22일,80.16
...,...
2020-2010-02월 07일,50.32
2020-2010-02월 10일,49.57
2020-2010-02월 11일,49.94


In [ ]:
pip install -U finance-datareader

In [ ]:
import FinanceDataReader as fdr
fdr.__version__

'0.9.6'

In [ ]:
ixic = fdr.DataReader('IXIC', '2010-02-14', '2020-02-13')
ixic.tail()

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2020-02-07,9520.51,9526.64,9570.09,9496.53,549790000.0,-0.0054
2020-02-10,9628.39,9493.63,9628.66,9493.63,546600000.0,0.0113
2020-02-11,9638.94,9680.89,9714.74,9617.21,567800000.0,0.0011
2020-02-12,9725.96,9688.60,9728.77,9666.68,589710000.0,0.0090
2020-02-13,9711.97,9657.04,9748.32,9650.02,545840000.0,-0.0014


In [ ]:
ixic.shape

(2517, 6)

In [ ]:
ixic = ixic[['Close']]
ixic

,Close
Date,
2010-02-16,2214.19
2010-02-17,2226.29
2010-02-18,2241.71
2010-02-19,2243.87
2010-02-22,2242.03
...,...
2020-02-07,9520.51
2020-02-10,9628.39
2020-02-11,9638.94


In [ ]:
dji = fdr.DataReader('DJI', '2010-02-14', '2020-02-13')
dji

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2010-02-16,10268.81,10100.81,10292.62,10100.81,0.0,0.0168
2010-02-17,10309.24,10261.48,10366.83,10223.16,0.0,0.0039
2010-02-18,10392.90,10309.39,10416.79,10255.58,0.0,0.0081
2010-02-19,10402.35,10387.77,10459.18,10316.50,0.0,0.0009
2010-02-22,10383.38,10402.43,10468.55,10333.05,0.0,-0.0018
...,...,...,...,...,...,...
2020-02-07,29102.51,29286.92,29286.92,29056.98,253870000.0,-0.0094
2020-02-10,29276.82,28995.66,29278.07,28995.66,257460000.0,0.0060
2020-02-11,29276.34,29390.71,29415.39,29210.47,281850000.0,-0.0000


In [ ]:
dji = dji[['Close']]
ixic.columns = ['NASDAQ']
dji.columns = ['DOW']


In [ ]:
ixic.index

DatetimeIndex(['2010-02-16', '2010-02-17', '2010-02-18', '2010-02-19',
               '2010-02-22', '2010-02-23', '2010-02-24', '2010-02-25',
               '2010-02-26', '2010-03-01',
               ...
               '2020-01-31', '2020-02-03', '2020-02-04', '2020-02-05',
               '2020-02-06', '2020-02-07', '2020-02-10', '2020-02-11',
               '2020-02-12', '2020-02-13'],
              dtype='datetime64[ns]', name='Date', length=2517, freq=None)

In [ ]:
tmp3 = pd.Series (ixic.index)
date_str = tmp3.apply( lambda x:str(x)[:10] )
date_str

0       2010-02-16
1       2010-02-17
2       2010-02-18
3       2010-02-19
4       2010-02-22
           ...    
2512    2020-02-07
2513    2020-02-10
2514    2020-02-11
2515    2020-02-12
2516    2020-02-13
Name: Date, Length: 2517, dtype: object

In [ ]:
ixic['Date'] = date_str.values

In [ ]:
ixic.set_index(['Date'], inplace=True)

In [ ]:
ixic.head()

,NASDAQ
Date,
2010-02-16,2214.19
2010-02-17,2226.29
2010-02-18,2241.71
2010-02-19,2243.87
2010-02-22,2242.03


In [ ]:
tmp3 = pd.Series (dji.index)
date_str = tmp3.apply( lambda x:str(x)[:10] )
dji['Date'] = date_str.values
dji.set_index(['Date'], inplace=True)
dji.head()

,DOW
Date,
2010-02-16,10268.81
2010-02-17,10309.24
2010-02-18,10392.90
2010-02-19,10402.35
2010-02-22,10383.38


In [ ]:
date1 = date & dollar.index.unique() & interest.index.unique() & oil_10years.index.unique() & dji.index.unique() & ixic.index.unique()

In [ ]:
print(date1)

Index([], dtype='object', name='Date')


In [ ]:
top10_10years[0].index

Index(['2010-02-16', '2010-02-17', '2010-02-18', '2010-02-19', '2010-02-22',
       '2010-02-23', '2010-02-24', '2010-02-25', '2010-02-26', '2010-03-02',
       ...
       '2020-01-31', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06',
       '2020-02-07', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13'],
      dtype='object', name='Date', length=2451)

In [ ]:
interest = interest.loc[date1, :]
oil_10years = oil_10years.loc[date1, :]
dji = dji.loc[date1, :]
dollar = dollar.loc[date1, :]
ixic = ixic.loc[date1, :]

In [ ]:
interest.shape, oil_10years.shape ,dji.shape, dollar.shape, ixic.shape

((0, 1), (0, 1), (0, 1), (0, 1), (0, 1))

In [ ]:
top7 = list()
for i in range(len(top10_10years)):
  top7.append(top10_10years[i].loc[date1, :])

top7[0].shape

(0, 1)

In [ ]:
top7[1]

,Close
Date,


In [ ]:
# 일진다이아 081000
# 디피씨 026890
# JW중외제약우 001065
# 현대건설우 000725
# 유니온머티리얼 047400
# 두산2우B 000157
# 모나미 005360
top7[0]['name'] = '일진다이아'
top7[0]['code'] = '081000'
top7[0].head()

,Close,name,code
Date,,,


In [ ]:
top7[1]

,Close
Date,


In [ ]:
top7[1]['name'] = '디피씨'
top7[1]['code'] = '026890'
top7[2]['name'] = 'JW중외제약우'
top7[2]['code'] = '001065'
top7[3]['name'] = '현대건설우'
top7[3]['code'] = '000725'
top7[4]['name'] = '유니온머티리얼'
top7[4]['code'] = '047400'
top7[5]['name'] = '두산2우B'
top7[5]['code'] = '000157'
top7[6]['name'] = '모나미'
top7[6]['code'] = '005360'

In [ ]:
top7[4]

,Close,name,code
Date,,,


In [ ]:
top7[2]

,Close,name,code
Date,,,


In [ ]:
top7_variable = list()
# interest.shape, oil_10years.shape ,dji.shape, dollar.shape, ixic.shape
temp1 = pd.merge(top7[0], dollar, left_index=True, right_index=True)
temp2 = pd.merge(temp1, oil_10years, left_index=True, right_index=True)
temp3 = pd.merge(temp2, ixic, left_index=True, right_index=True)
temp4 = pd.merge(temp3, dji, left_index=True, right_index=True)
top7_variable.append(pd.merge(temp4, interest, left_index=True, right_index=True))
top7_variable[1]

IndexError: ignored

In [ ]:
top7_variable = list()
for i in range(len(top7)):
  temp1 = pd.merge(top7[i], dollar, left_index=True, right_index=True)
  temp2 = pd.merge(temp1, oil_10years, left_index=True, right_index=True)
  temp3 = pd.merge(temp2, ixic, left_index=True, right_index=True)
  temp4 = pd.merge(temp3, dji, left_index=True, right_index=True)
  top7_variable.append(pd.merge(temp4, interest, left_index=True, right_index=True))


In [ ]:
top7_variable[6]

,Close,name,code,dollar,WTI,NASDAQ,DOW,interest_rate
Date,,,,,,,,


In [ ]:
for i in range(len(top7_variable)):
  top7_variable[i] = top7_variable[i][[ 'name', 	'code',  'Close','dollar', 'WTI',	'NASDAQ',	'DOW',	'interest_rate']]

In [ ]:
top7_variable[5]

,name,code,Close,dollar,WTI,NASDAQ,DOW,interest_rate
Date,,,,,,,,


In [ ]:
top7_variable[0].columns[3:]

Index(['dollar', 'WTI', 'NASDAQ', 'DOW', 'interest_rate'], dtype='object')

In [ ]:
top7_variable[0].dtypes

name              object
code              object
Close            float64
dollar           float64
WTI               object
NASDAQ           float64
DOW              float64
interest_rate     object
dtype: object

In [ ]:
top7_variable[0]['interest_rate'] = top7_variable[0]['interest_rate'].apply(lambda x : float(x))
top7_variable[0].dtypes

name              object
code              object
Close            float64
dollar           float64
WTI               object
NASDAQ           float64
DOW              float64
interest_rate     object
dtype: object

In [ ]:
for i in range(1, len(top7_variable)):
  top7_variable[i]['WTI'] = top7_variable[i]['WTI'].apply(lambda x : float(x))
  top7_variable[i]['interest_rate'] = top7_variable[i]['interest_rate'].apply(lambda x : float(x))
top7_variable[2].dtypes

name              object
code              object
Close            float64
dollar           float64
WTI               object
NASDAQ           float64
DOW              float64
interest_rate     object
dtype: object

In [ ]:
for i in range(len(top7_variable)):
  top7_variable[i].dropna(axis=0, inplace=True)

In [6]:
tmp = pd.read_csv('/content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/10years.csv', encoding='utf-8')
tmp

,Date,name,code,Close,dollar,WTI,NASDAQ,DOW,interest_rate
0,2010-02-17,일진다이아,81000,1773.380005,1152.9,77.33,2226.29,10309.24,2.01
1,2010-02-18,일진다이아,81000,1773.380005,1142.7,79.06,2241.71,10392.90,2.01
2,2010-02-19,일진다이아,81000,1773.380005,1147.5,79.81,2243.87,10402.35,2.01
3,2010-02-22,일진다이아,81000,1779.199951,1157.8,80.16,2242.03,10383.38,1.98
4,2010-02-23,일진다이아,81000,1769.500000,1149.3,78.86,2213.44,10282.41,1.92
...,...,...,...,...,...,...,...,...,...
15932,2020-02-07,모나미,5360,3205.000000,1181.4,50.32,9520.51,29102.51,1.22
15933,2020-02-10,모나미,5360,3205.000000,1187.2,49.57,9628.39,29276.82,1.24
15934,2020-02-11,모나미,5360,3225.000000,1189.8,49.94,9638.94,29276.34,1.26
15935,2020-02-12,모나미,5360,4190.000000,1184.1,51.17,9725.96,29551.42,1.26


In [ ]:
len(tmp)

In [7]:
# 0:2276
# 2276:4553
# 4553:6830
# 6830:9107
# 9107:11384
# 11384:13660
# 13660:15937
top7_variable = list()
top7_variable.append(tmp[0:2276])
top7_variable.append(tmp[2276:4553])
top7_variable.append(tmp[4553:6830])
top7_variable.append(tmp[6830:9107])
top7_variable.append(tmp[9107:11384])
top7_variable.append(tmp[11384:13660])
top7_variable.append(tmp[13660:15937])
top7_variable[6]

,Date,name,code,Close,dollar,WTI,NASDAQ,DOW,interest_rate
13660,2010-02-17,모나미,5360,1555.930054,1152.9,77.33,2226.29,10309.24,2.01
13661,2010-02-18,모나미,5360,1555.930054,1142.7,79.06,2241.71,10392.90,2.01
13662,2010-02-19,모나미,5360,1555.930054,1147.5,79.81,2243.87,10402.35,2.01
13663,2010-02-22,모나미,5360,1565.160034,1157.8,80.16,2242.03,10383.38,1.98
13664,2010-02-23,모나미,5360,1560.540039,1149.3,78.86,2213.44,10282.41,1.92
...,...,...,...,...,...,...,...,...,...
15932,2020-02-07,모나미,5360,3205.000000,1181.4,50.32,9520.51,29102.51,1.22
15933,2020-02-10,모나미,5360,3205.000000,1187.2,49.57,9628.39,29276.82,1.24
15934,2020-02-11,모나미,5360,3225.000000,1189.8,49.94,9638.94,29276.34,1.26
15935,2020-02-12,모나미,5360,4190.000000,1184.1,51.17,9725.96,29551.42,1.26


In [8]:
for i in range(len(top7_variable)):
  top7_variable[i].set_index(['Date'], inplace=True)

In [9]:
top7_variable[0]['name']['2010-02-18']

'일진다이아'

In [10]:
cols = ['interest_rate',	'dollar',	'WTI',	'NASDAQ',	'DOW',	'Close']
columns = top7_variable[0].columns[3:]

In [15]:


for i in range(len(top7_variable)):
  print('#'*55)
  print('#'*25, top7_variable[i]['name']['2010-02-18'], '#'*25)
  print('#'*55)
  sns.set(style='whitegrid')
  sns.pairplot(top7_variable[i][cols], height=2.5)
  plt.show()
  corr_tmp = np.corrcoef(top7_variable[i][cols].values.T)
  sns.heatmap(corr_tmp, cmap='RdYlGn_r', cbar=True, annot=True, square=True, fmt='.2f',
              annot_kws={'size':15}, yticklabels=cols, xticklabels=cols)
  plt.show()
  target_df = top7_variable[i][columns]
  y_data = top7_variable[i]['Close']
  X_train, X_test, y_train, y_test = train_test_split(target_df, y_data, train_size=0.7, 
                                                      random_state=42)
  X_train.shape, X_test.shape, y_train.shape, y_test.shape
  X_train_added = sm.add_constant(X_train)
  full_model= sm.OLS(y_train, X_train_added)
  full_model_res = full_model.fit()
  print(full_model_res.summary())
  print(f'{"인덱스":20} {"R제곱값":10} {"다중공선성":10}')
  print('-'*60)
  # r제곱이 가장 컸을때
  best_r_idx  = -1
  best_rsqrd  = -1
  best_vif    = -1
  best_n1     = 0
  best_v_idx  = -1
  best_rsqrd2 = -1
  best_vif2   = 10
  best_n2     = 0
  for n in range(2, len(columns)):
    five_cols = [item for item in list(combinations(columns, n ))]
    for idx, col in enumerate(five_cols):
      m_model = sm.OLS( y_train, sm.add_constant(X_train[list(col)]))
      m_model_res = m_model.fit()
      vif = 1/(1-m_model_res.rsquared)
      if m_model_res.rsquared > best_rsqrd:
        best_rsqrd = m_model_res.rsquared
        best_r_idx = idx
        best_vif  = vif
        best_n1 = n

      if vif < best_vif2:
        best_v_idx  = idx
        best_rsqrd2 = m_model_res.rsquared
        best_vif2   = vif
        best_n2 = n
    
  # 결과 
  print("R제곱 기준", best_r_idx, best_rsqrd, best_vif, best_n1)
  print("VIF 기준", best_v_idx, best_rsqrd2, best_vif2, best_n2)
  print("최고의 조합 : ", five_cols[best_r_idx])

  m_model = sm.OLS( y_train, sm.add_constant(X_train[list(five_cols[best_r_idx])]))
  m_model_res = m_model.fit()
  target_cols = list(five_cols[best_r_idx])
  # 예측할때 데이터도 동일하게 맞춰준다
  y_pred = m_model_res.predict( sm.add_constant(X_test[target_cols]))
  print(y_pred)
  print("성능평가", r2_score( y_test, y_pred ), m_model_res.rsquared)
  plt.scatter(y_test, y_pred, alpha=0.5)
  plt.xlabel("Actual Rent")
  plt.ylabel("Predicted Rent")
  plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
top7_variable[0].shape, top7_variable[1].shape, top7_variable[2].shape, top7_variable[3].shape, top7_variable[4].shape, top7_variable[5].shape, top7_variable[6].shape

In [ ]:
2276*7

In [ ]:
df = pd.concat([top7_variable[0], top7_variable[1], top7_variable[2], top7_variable[3], top7_variable[4], top7_variable[5], top7_variable[6]])

In [ ]:
df.to_csv('/content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/10years.csv', encoding='utf-8')